# Проект по SQL

## Задачи

Главная цель — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
query = '''SELECT * 
           FROM books 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT * 
           FROM authors 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * 
           FROM publishers 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''SELECT * 
           FROM ratings 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''SELECT * 
           FROM reviews 
           LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


1) Посчитайте, сколько книг вышло после 1 января 2000 года.

In [8]:
query = '''SELECT COUNT(*) 
           FROM books 
           WHERE publication_date > '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Всего после 1 января 2000 года вышло 819 книг.

2) Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [9]:
query = ''' 
WITH
a AS (SELECT book_id,
             COUNT(review_id) as count_review
             FROM reviews
             GROUP BY book_id),
             
c AS (SELECT book_id,
             AVG(rating) as avg_rating
             FROM ratings
             GROUP BY book_id)

SELECT title,
       a.count_review,
       c.avg_rating
FROM books AS b LEFT JOIN a ON b.book_id=a.book_id
LEFT JOIN c ON b.book_id=c.book_id
ORDER BY c.avg_rating DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,title,count_review,avg_rating
0,Pop Goes the Weasel (Alex Cross #5),2.0,5.00
1,The Ghost Map: The Story of London's Most Terr...,2.0,5.00
2,In the Hand of the Goddess (Song of the Liones...,2.0,5.00
3,Tai-Pan (Asian Saga #2),2.0,5.00
4,How to Be a Domestic Goddess: Baking and the A...,1.0,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,Junky,2.0,2.00
997,His Excellency: George Washington,2.0,2.00
998,Drowning Ruth,3.0,2.00


Самый низкий рейтинг - 1.5, самый большой, соответственно, - 5.

3) Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query = '''SELECT publisher,
           COUNT(b.book_id) AS count_books
           FROM publishers AS p LEFT JOIN books AS b ON p.publisher_id=b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY publisher 
           ORDER BY count_books DESC
           LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


Penguin Books - самое крупное по число выпущенных книг (толще 50 страниц) издательство.

4) Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [11]:
query = '''
WITH
tb1 AS (SELECT rat.book_id,
             AVG(rat.rating) AS avg_rating,
             COUNT(rat.rating) AS count_rating
      FROM ratings AS rat 
      GROUP BY rat.book_id
      HAVING COUNT(rating) >=50),
      
tb2 AS (SELECT a.author_id,
             SUM(tb1.count_rating) AS total_rating,
             AVG(tb1.avg_rating) AS avg_rating
       FROM authors AS a JOIN books b ON a.author_id = b.author_id
       JOIN tb1 ON b.book_id = tb1.book_id
       GROUP BY a.author_id)

SELECT a.author,
       tb2.total_rating,
       tb2.avg_rating
FROM author AS a JOIN tb2 ON a.author_id=tb2.author_id
ORDER BY tb2.total_rating DESC
LIMIT 1          
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,total_rating,avg_rating
0,J.K. Rowling/Mary GrandPré,310.0,4.283844


Дж. К. Роулин - писательница с самой высокой средней оценкой книг (учитывали книги с 50 и более оценками).

5) Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
WITH
a AS (SELECT username
      FROM ratings
      GROUP BY username
      HAVING COUNT(rating_id) > 48),

b AS (SELECT COUNT (review_id) AS count_review,
             username AS name
      FROM reviews 
      GROUP BY username),

m AS (SELECT b.count_review,
             a.username
      FROM a LEFT JOIN b ON a.username=b.name)

SELECT AVG(m.count_review)
FROM m
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, поставивших больше 48 оценок - 24

## Общий вывод

Исходя из анализаа данных, можно сделать следующие выводы:

1) Всего после 1 января 2000 года вышло 819 книг (всего представлено в датасете 1000 книг).

2) Самый низкий рейтинг - 1.5, самый большой, соответственно, - 5.

3) Penguin Books - самое крупное по число выпущенных книг (толще 50 страниц) издательство.

4) Дж. К. Роулин - писательница с самой высокой средней оценкой книг (учитывали книги с 50 и более оценками).

5) Среднее количество обзоров от пользователей, поставивших больше 48 оценок - 24.